# Experimenting with Prompts

We will try experimenting with prompts for the following types of models:
1. Vanilla whisper
2. HF whisper
3. Whisper C++
4. faster-whisper


In [1]:
import whisper

model = whisper.load_model("turbo")

100%|█████████████████████████████████████| 1.51G/1.51G [01:12<00:00, 22.2MiB/s]


In [10]:
result_base = model.transcribe("./atc_test.mp3",language="english")

general = ['Air Traffic Control communications', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '90', '180', '270',
           '360']
nato = ['Alpha', 'Bravo', 'Charlie', 'Delta', 'Echo', 'Foxtrot', 'Golf', 'Hotel', 'India', 'Juliett', 'Kilo', 'Lima',
        'Mike', 'November', 'Oscar', 'Papa', 'Quebec', 'Romeo', 'Sierra', 'Tango', 'Uniform', 'Victor', 'Whiskey',
        'Xray', 'Yankee', 'Zulu']
relevant_atc_phrases = [
    "hold short",
    "hold position",
    "standby",
    "go-ahead",
    "go around",
    "missed approach",
    "climb to",
    "maintain",
    "contact",
    "departure",
    "frequency",
    "tower",
    "runway",
    "ground",
    "roger",
    "readback",
    "position",
    "report",
    "information",
    "altitude",
]


collated_list = general + nato + relevant_atc_phrases
collated_list_string = ' '.join(collated_list)

result_prompt = model.transcribe("./atc_test.mp3", initial_prompt=collated_list_string, language="english")
print(result_base["text"])
print(result_prompt['text'])

/opt/miniconda3/envs/atlas/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/opt/miniconda3/envs/atlas/lib/python3.13/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Singapore 638, can we stop here for the moment? Singapore 638, second please Singapore 638, we like to stop here at the moment Singapore 638, okay, hold the foot short Okay, holding short, now I'm stopping here now, Singapore 638, thank you Singapore 638, any smoke coming out from our tyres? I reject your standby Thank you Tower, good morning VJ822, 3 miles, 16 left BJ822, Narita Tower, runway 16 left closed 22 runway inspection Go around, execute missed approach, climb and maintain, 3 times Copy, 3000, go around VJ822 Tower VJ822, climbing to 3000, missed approach, 16 left VJ822, correct, missed approach, maintain 3000 Copy, maintain 3000, VJ822 Singapore 638, enter Singapore 638, go ahead Singapore 638, hold present position, vehicle will check your landing gear from your forward Roger, Singapore 638, hold present position Roger, Singapore 638, hold present position VJ822, contact Tokyo departure, 124.2 124.2, confirm VJ822 Correct, 124.2 for further instruction Copy, 124.2, VJ822 L

In [11]:
print(result_base["text"])


 Singapore 638, can we stop here for the moment? Singapore 638, second please Singapore 638, we like to stop here at the moment Singapore 638, okay, hold the foot short Okay, holding short, now I'm stopping here now, Singapore 638, thank you Singapore 638, any smoke coming out from our tyres? I reject your standby Thank you Tower, good morning VJ822, 3 miles, 16 left BJ822, Narita Tower, runway 16 left closed 22 runway inspection Go around, execute missed approach, climb and maintain, 3 times Copy, 3000, go around VJ822 Tower VJ822, climbing to 3000, missed approach, 16 left VJ822, correct, missed approach, maintain 3000 Copy, maintain 3000, VJ822 Singapore 638, enter Singapore 638, go ahead Singapore 638, hold present position, vehicle will check your landing gear from your forward Roger, Singapore 638, hold present position Roger, Singapore 638, hold present position VJ822, contact Tokyo departure, 124.2 124.2, confirm VJ822 Correct, 124.2 for further instruction Copy, 124.2, VJ822 L

In [12]:
print(result_prompt['text'])


 Singapore 638 can we stop here for the moment? Singapore 638 second please Singapore 638 we like to stop here at the moment Singapore 638 okay hold the position Okay holding short now I'm stopping here now Singapore 638 thank you Singapore 638 any smoke coming out from our tyres? I reject your standby Thank you Tower good morning VJ822 3 miles 16 left BJ822 Narita Tower runway 16 left close 22 runway inspection go around execute missed approach climb and maintain 3 times Copy 3000 go around VJ822 Tower VJ822 climbing to 3000 missed approach 16 left BJ822 correct miss approach Maintain 3000 Copy maintain 3000 VJ822 SINGAPOR 6 later SINGAPOR 638 go ahead SINGAPOR 638 hold present position vehicle will check your landing gear from your forward Roger SINGAPOR 638 holding position BJ822 contact TOKYO DEPANCHER 124.2 BJ822 contact TOKYO DEPANCHER 124.2 124.2 124.2 confirm VJ822 Correct 124.2 for further instruction Copy say 124.2 VJ822 SINGAPOR 638 SINGAPOR 638 go ahead Vehicle check your l

In [25]:
import re
from num2words import num2words

def load_text(file_path):
    """Load text from a file and return it as a string."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def normalize_text(text):
    """Normalize the text: lowercase, remove punctuation, convert numbers, flatten whitespace."""
    
    # Lowercase
    text = text.lower()

    # Function to replace digit sequences with spelled-out words (digit by digit)
    def replace_digits(match):
        digits = match.group()
        return ' '.join(num2words(int(d)).replace('-', ' ') for d in digits)

    # Replace all digit sequences, even inside alphanumeric words
    text = re.sub(r'\d+', replace_digits, text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Normalize whitespace and flatten to one line
    text = ' '.join(text.split())

    return text


raw_text = load_text('./ground_truth.txt')
ground_truth = normalize_text(raw_text)
print(ground_truth)


singapore six three eight can we stop here for a moment singapore six three eight say again please singapore six three eight wed like to stop here stop here at the moment singapore six three eight okay hold double short okay holding short now stopping here now singapore six three eight singapore six three eight any smoke coming out from our tyres i will check standby tower good morning vietjet eight two two three miles one six left vietjet eight two two narita tower runway one six left closed due to runway inspection go around execute missed approach climb and maintain three zero zero zero copy three zero zero zero go around vietjet eight two two tower vietjet eight two two climbing to three zero zero zero missed approach one six left vietjet eight two two correct missed approach maintain three zero zero zero copy maintain three zero zero zero vietjet eight two two singapore six three eight tower singapore six three eight go ahead singapore six three eight hold present position vehicle

In [26]:
result_base_text = normalize_text(result_base["text"])
result_prompt_text = normalize_text(result_prompt["text"]).replace('singapor', 'singapore')

print('base:\n',result_base_text)
print('prompt:\n',result_prompt_text)

base:
 singapore six three eight can we stop here for the moment singapore six three eight second please singapore six three eight we like to stop here at the moment singapore six three eight okay hold the foot short okay holding short now im stopping here now singapore six three eight thank you singapore six three eight any smoke coming out from our tyres i reject your standby thank you tower good morning vjeight two two three miles one six left bjeight two two narita tower runway one six left closed two two runway inspection go around execute missed approach climb and maintain three times copy three zero zero zero go around vjeight two two tower vjeight two two climbing to three zero zero zero missed approach one six left vjeight two two correct missed approach maintain three zero zero zero copy maintain three zero zero zero vjeight two two singapore six three eight enter singapore six three eight go ahead singapore six three eight hold present position vehicle will check your landin

In [50]:
from jiwer import wer

# Calculate WER for both base and prompt outputs
wer_base = wer(ground_truth, result_base_text)
wer_prompt = wer(ground_truth, result_prompt_text)

# print(f"WER (Base Model): {wer_base:.2%}")
# print(f"WER (Prompted Model): {wer_prompt:.2%}")

print(f"Accuracy (Base Model): {(1-wer_base):.2%}")
print(f"Accuracy (Prompted Model): {(1-wer_prompt):.2%}")


Accuracy (Base Model): 59.47%
Accuracy (Prompted Model): 77.43%
